## 15. 사전 훈련된 워드 임베딩
### 15.1 케라스의 임베딩 층(embedding layer) 사용
- 훈련 데이터의 단어들에 대해 워드 임베딩을 수행하는 Embedding() 도구 제공
- v = Embedding(20000, 128, input_length=500) / vocab_size = 20000 / output_dim = 128 /input_length = 500
- vocab_size : 텍스트 데이터의 전체 단어 집합의 크기 / output_dim : 워드 임베딩 후의 임베딩 벡터의 차원 / input_length : 입력 시퀀스의 길이

### 15.2 사전 훈련된 워드 임베딩(pre-trained word embedding) 사용
- 위키피디아 등과 같은 방대한 코퍼스를 가지고 Word2vec, FastText, GloVe 등을 통해서 이미 미리 훈련된 임베딩 벡터를 불러오는 방법
- 임베딩 층은 룩업 테이블
    - 입력 정수에 대해 밀집 벡터(dense vector)로 맵핑
    - 이 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련
    - 훈련 과정에서 단어는 모델이 풀고자하는 작업에 맞는 값으로 업데이트

![embedding_vector](img/embedding_vector.png)

### 15.3 문장의 긍,부정을 판단하는 감성 분석 모델 - Keras

In [2]:
# 문장의 긍, 부정을 판단하는 감성 분류 모델
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# (1) 문장과 레이블 데이터
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

# (2) keras token화
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1
print(vocab_size)

# (3) 정수인코딩 수행
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

# (4) 가장 긴문장 체크
max_len=max(len(l) for l in X_encoded)
print(max_len)

# (5) 가장 긴 문장의 길이로 맞춰줌 - padding
X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train=np.array(y_train)
print(X_train)

16
[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]
4
[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [3]:
# (6) 학습
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len)) # 모든 임베딩 벡터는 4차원.
model.add(Flatten()) # Dense의 입력으로 넣기위함.
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=20, verbose=2)

Epoch 1/20
1/1 - 0s - loss: 0.6912 - acc: 0.4286
Epoch 2/20
1/1 - 0s - loss: 0.6895 - acc: 0.4286
Epoch 3/20
1/1 - 0s - loss: 0.6877 - acc: 0.4286
Epoch 4/20
1/1 - 0s - loss: 0.6860 - acc: 0.4286
Epoch 5/20
1/1 - 0s - loss: 0.6843 - acc: 0.4286
Epoch 6/20
1/1 - 0s - loss: 0.6826 - acc: 0.5714
Epoch 7/20
1/1 - 0s - loss: 0.6808 - acc: 0.5714
Epoch 8/20
1/1 - 0s - loss: 0.6791 - acc: 0.5714
Epoch 9/20
1/1 - 0s - loss: 0.6774 - acc: 0.5714
Epoch 10/20
1/1 - 0s - loss: 0.6757 - acc: 0.5714
Epoch 11/20
1/1 - 0s - loss: 0.6739 - acc: 0.5714
Epoch 12/20
1/1 - 0s - loss: 0.6722 - acc: 0.5714
Epoch 13/20
1/1 - 0s - loss: 0.6705 - acc: 0.5714
Epoch 14/20
1/1 - 0s - loss: 0.6687 - acc: 0.5714
Epoch 15/20
1/1 - 0s - loss: 0.6670 - acc: 0.7143
Epoch 16/20
1/1 - 0s - loss: 0.6653 - acc: 0.7143
Epoch 17/20
1/1 - 0s - loss: 0.6635 - acc: 0.8571
Epoch 18/20
1/1 - 0s - loss: 0.6618 - acc: 0.8571
Epoch 19/20
1/1 - 0s - loss: 0.6600 - acc: 1.0000
Epoch 20/20
1/1 - 0s - loss: 0.6583 - acc: 1.0000


### 15.4 사전 훈련된 워드 임베딩(Pre-Trained Word Embedding) 사용
- 훈련 데이터가 적다면 케라스의 Embedding()으로 해당 문제에 충분히 특화된 임베딩 벡터를 만들어내는 것이 쉽지 않음
- 훈련 데이터가 적은 상황이라면 모델에 케라스의 Embedding()을 사용하는 것보다 다른 텍스트 데이터로 사전 훈련되어 있는 임베딩 벡터를 불러오는 것이 나은 선택
- Word2Vec이나 GloVe 등으로 학습되어져 있는 임베딩 벡터들을 사용하는 것이 성능의 개선 가능
- Glove 다운로드 링크 : http://nlp.stanford.edu/data/glove.6B.zip
- Word2Vec 다운로드 링크 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM



### 15.4.1 사전 훈련된 GloVe 사용
- glove.6B.100d.txt는 수많은 단어에 대해서 100개의 차원을 가지는 임베딩 벡터로 제공
- 위의 출력 결과는 단어 'the'에 대해서 100개의 차원을 가지는 임베딩 벡터와 단어 ','에 대해서 100개의 차원을 가지는 임베딩 벡터

In [14]:

# glove.6B.100d.txt는 수많은 단어에 대해서 100개의 차원을 가지는 임베딩 벡터로 제공
n=0
f = open('glove.6B/glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split() # 각 줄을 읽어와서 word_vector에 저장.
    print(word_vector) # 각 줄을 출력
    word = word_vector[0] # word_vector에서 첫번째 값만 저장
    print(word) # word_vector의 첫번째 값만 출력
    n=n+1
    if n==2:
        break
f.close()

print(type(word_vector))
print(len(word_vector))

['the', '-0.038194', '-0.24487', '0.72812', '-0.39961', '0.083172', '0.043953', '-0.39141', '0.3344', '-0.57545', '0.087459', '0.28787', '-0.06731', '0.30906', '-0.26384', '-0.13231', '-0.20757', '0.33395', '-0.33848', '-0.31743', '-0.48336', '0.1464', '-0.37304', '0.34577', '0.052041', '0.44946', '-0.46971', '0.02628', '-0.54155', '-0.15518', '-0.14107', '-0.039722', '0.28277', '0.14393', '0.23464', '-0.31021', '0.086173', '0.20397', '0.52624', '0.17164', '-0.082378', '-0.71787', '-0.41531', '0.20335', '-0.12763', '0.41367', '0.55187', '0.57908', '-0.33477', '-0.36559', '-0.54857', '-0.062892', '0.26584', '0.30205', '0.99775', '-0.80481', '-3.0243', '0.01254', '-0.36942', '2.2167', '0.72201', '-0.24978', '0.92136', '0.034514', '0.46745', '1.1079', '-0.19358', '-0.074575', '0.23353', '-0.052062', '-0.22044', '0.057162', '-0.15806', '-0.30798', '-0.41625', '0.37972', '0.15006', '-0.53212', '-0.2055', '-1.2526', '0.071624', '0.70565', '0.49744', '-0.42063', '0.26148', '-1.538', '-0.30223

In [16]:
import numpy as np
embedding_dict = dict()
f = open('glove.6B/glove.6B.100d.txt', encoding="utf8")

# 형식은 키(key)와 값(value)의 쌍(pair)를 가지는 파이썬의 사전형 구조를 사용
for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))
embedding_matrix = np.zeros((vocab_size, 100))
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
print(np.shape(embedding_matrix))
print(t.word_index.items())

# 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입
        
    

400000개의 Embedding vector가 있습니다.
[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

# 현재 실습에서 사전 훈련된 워드 임베딩을 100차원의 값인 것으로 사용하고 있기 때문에 임베딩 층의 output_dim의 인자값으로 100
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=10, verbose=2)

Epoch 1/10
1/1 - 0s - loss: 0.6051 - acc: 0.5714
Epoch 2/10
1/1 - 0s - loss: 0.5875 - acc: 0.7143
Epoch 3/10
1/1 - 0s - loss: 0.5706 - acc: 0.7143
Epoch 4/10
1/1 - 0s - loss: 0.5544 - acc: 0.7143
Epoch 5/10
1/1 - 0s - loss: 0.5388 - acc: 0.7143
Epoch 6/10
1/1 - 0s - loss: 0.5239 - acc: 0.7143
Epoch 7/10
1/1 - 0s - loss: 0.5095 - acc: 0.7143
Epoch 8/10
1/1 - 0s - loss: 0.4957 - acc: 0.8571
Epoch 9/10
1/1 - 0s - loss: 0.4825 - acc: 0.8571
Epoch 10/10
1/1 - 0s - loss: 0.4698 - acc: 0.8571


### 15.4.1 사전 훈련된 Word2Vec 사용
- glove.6B.100d.txt는 수많은 단어에 대해서 100개의 차원을 가지는 임베딩 벡터로 제공
- 위의 출력 결과는 단어 'the'에 대해서 100개의 차원을 가지는 임베딩 벡터와 단어 ','에 대해서 100개의 차원을 가지는 임베딩 벡터
- 링크 : https://wikidocs.net/33793
- https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
